In [38]:
# 导入必要的库
import os
import shutil
import random
from pathlib import Path
from typing import List, Tuple
from tqdm import tqdm
def find_annotated_images(image_dir: str, annotation_dir: str, image_extensions: List[str] = ['.jpg','.tiff', '.jpeg', '.png', '.bmp']) -> Tuple[List[Path], List[Path]]:
    """
    查找有标注和无标注的图片。

    Args:
        image_dir (str): 图片目录路径。
        annotation_dir (str): 标注文件目录路径。
        image_extensions (List[str], optional): 支持的图片扩展名列表。默认为 ['.jpg', '.jpeg', '.png', '.bmp']。

    Returns:
        Tuple[List[Path], List[Path]]: 有标注的图片路径列表和无标注的图片路径列表。
    """
    image_dir = Path(image_dir)
    annotation_dir = Path(annotation_dir)

    # 收集所有图片文件
    all_images = [p for p in image_dir.iterdir() if p.suffix.lower() in image_extensions and p.is_file()]

    annotated_images = []
    non_annotated_images = []

    for img_path in tqdm(all_images, desc="查找有标注和无标注的图片"):
        # 假设标注文件与图片同名但扩展名为 .txt
        label_path = annotation_dir / f"{img_path.stem}.json"
        if label_path.exists():
            annotated_images.append(img_path)
        else:
            non_annotated_images.append(img_path)

    return annotated_images, non_annotated_images

def split_data(annotated_images: List[Path], train_ratio: float = 0.8) -> Tuple[List[Path], List[Path]]:
    """
    将有标注的图片随机划分为训练集和验证集。

    Args:
        annotated_images (List[Path]): 有标注的图片路径列表。
        train_ratio (float, optional): 训练集比例。默认为 0.8。

    Returns:
        Tuple[List[Path], List[Path]]: 训练集图片路径列表和验证集图片路径列表。
    """
    random.shuffle(annotated_images)
    split_index = int(len(annotated_images) * train_ratio)
    train_images = annotated_images[:split_index]
    val_images = annotated_images[split_index:]
    return train_images, val_images

def organize_data(
    image_dir: str,
    annotation_dir: str,
    output_dir: str,
    annotated_images: List[Path],
    non_annotated_images: List[Path],
    train_ratio: float = 0.8
) -> None:
    """
    将图片和标注文件组织到不同的目录中，并划分训练集和验证集。

    Args:
        image_dir (str): 图片目录路径。
        annotation_dir (str): 标注文件目录路径。
        output_dir (str): 输出的组织后数据的目录路径。
        annotated_images (List[Path]): 有标注的图片路径列表。
        non_annotated_images (List[Path]): 无标注的图片路径列表。
        train_ratio (float, optional): 训练集比例。默认为 0.8。
    """
    image_dir = Path(image_dir)
    annotation_dir = Path(annotation_dir)
    output_dir = Path(output_dir)

    train_images_dir = output_dir / 'train' / 'images'
    train_labels_dir = output_dir / 'train' / 'labels'
    val_images_dir = output_dir / 'val' / 'images'
    val_labels_dir = output_dir / 'val' / 'labels'
    non_annotated_images_dir = output_dir / 'non_annotated_images'

    # 创建目标目录
    train_images_dir.mkdir(parents=True, exist_ok=True)
    train_labels_dir.mkdir(parents=True, exist_ok=True)
    val_images_dir.mkdir(parents=True, exist_ok=True)
    val_labels_dir.mkdir(parents=True, exist_ok=True)
    non_annotated_images_dir.mkdir(parents=True, exist_ok=True)

    # 划分训练集和验证集
    train_images, val_images = split_data(annotated_images, train_ratio)

    # 复制训练集
    for img_path in tqdm(train_images, desc="复制训练集图片"):
        shutil.copy(img_path, train_images_dir / img_path.name)
        label_path = annotation_dir / f"{img_path.stem}.json"
        if label_path.exists():
            shutil.copy(label_path, train_labels_dir / label_path.name)

    # 复制验证集
    for img_path in tqdm(val_images, desc="复制验证集图片"):
        shutil.copy(img_path, val_images_dir / img_path.name)
        label_path = annotation_dir / f"{img_path.stem}.json"
        if label_path.exists():
            shutil.copy(label_path, val_labels_dir / label_path.name)

    # 复制无标注的图片
    for img_path in tqdm(non_annotated_images, desc="复制无标注的图片"):
        shutil.copy(img_path, non_annotated_images_dir / img_path.name)

    print(f"训练集图片已复制到: {train_images_dir}")
    print(f"训练集标注文件已复制到: {train_labels_dir}")
    print(f"验证集图片已复制到: {val_images_dir}")
    print(f"验证集标注文件已复制到: {val_labels_dir}")
    print(f"无标注的图片已复制到: {non_annotated_images_dir}")


# 设置目录路径
image_dir = "/data/yan/track/TS-20241123222043114.avi_frames"          # 原始图片目录
annotation_dir = "/data/yan/track/TS-20241123222043114.avi_frames/annotations"  # 标注文件目录
output_dir = "./dataset"                                 # 输出数据集目录
train_ratio = 0.8                                        # 训练集比例

# 查找有标注和无标注的图片
annotated_images, non_annotated_images = find_annotated_images(image_dir, annotation_dir)

print(f"找到 {len(annotated_images)} 张有标注的图片。")
print(f"找到 {len(non_annotated_images)} 张无标注的图片。")

# 组织数据并划分训练集和验证集
organize_data(
    image_dir=image_dir,
    annotation_dir=annotation_dir,
    output_dir=output_dir,
    annotated_images=annotated_images,
    non_annotated_images=non_annotated_images,
    train_ratio=train_ratio
)


查找有标注和无标注的图片: 100%|██████████| 2184/2184 [00:00<00:00, 163861.69it/s]


找到 23 张有标注的图片。
找到 2161 张无标注的图片。


复制无标注的图片:   0%|          | 0/2161 [00:00<?, ?it/s]

复制无标注的图片: 100%|██████████| 2161/2161 [00:01<00:00, 1140.36it/s]

训练集图片已复制到: dataset/train/images
训练集标注文件已复制到: dataset/train/labels
验证集图片已复制到: dataset/val/images
验证集标注文件已复制到: dataset/val/labels
无标注的图片已复制到: dataset/non_annotated_images


In [ ]:
import json
from pathlib import Path
from tqdm import tqdm

def convert_labelme_to_yolo_segmentation(json_dir: str, output_dir: str, classes: list):
    """
    将 Labelme 的 JSON 格式转化为 YOLO 分割格式。

    Args:
        json_dir (str): 存放 JSON 文件的目录。
        output_dir (str): 存放 YOLO 格式标注文件的目录。
        classes (list): 类别名称列表。
    """
    json_dir = Path(json_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    json_files = list(json_dir.glob("*.json"))
    
    for json_file in tqdm(json_files, desc="转换分割标注文件"):
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        img_width = data['imageWidth']
        img_height = data['imageHeight']
        yolo_annotations = []
        
        for shape in data['shapes']:
            label = shape['label']
            points = shape['points']
            
            # 归一化多边形点
            normalized_points = [
                f"{point[0] / img_width:.6f} {point[1] / img_height:.6f}" 
                for point in points
            ]
            
            if label in classes:
                class_id = classes.index(label)
                # YOLO 分割格式：<class_id> <x1> <y1> <x2> <y2> ... <xn> <yn>
                annotation = f"{class_id} " + " ".join(normalized_points)
                yolo_annotations.append(annotation)
            else:
                print(f"警告: 标签 {label} 不在 classes 列表中，已跳过。")
        
        # 输出 YOLO 格式的标注文件
        output_path = output_dir / f"{json_file.stem}.txt"
        with open(output_path, "w", encoding="utf-8") as f:
            f.write("\n".join(yolo_annotations))
    
    print(f"所有分割标注文件已转换并保存到 {output_dir}")

# 设置目录路径和类别列表
json_dir = "./dataset/train/labels"  # 标注文件 JSON 目录
output_dir = "./dataset/train/yololabels"  # 转换后保存 YOLO 格式的目录
classes = ["Celegans"]  # 替换为你的实际类别名称

# 转换标注文件
convert_labelme_to_yolo_segmentation(json_dir, output_dir, classes)
#  pip install ultralytics -i https://pypi.tuna.tsinghua.edu.cn/simple

转换分割标注文件: 100%|██████████| 18/18 [00:00<00:00, 334.06it/s]

所有分割标注文件已转换并保存到 dataset/train/yololabels


In [4]:
from ultralytics import YOLO

# 加载预训练的 YOLO11 分割模型
model = YOLO('/data/yan/track/yolomodel/yolo11x.pt')  # 确保模型支持分割任务

# 开始训练
model.train(
    data='./data.yaml',       # 数据集配置文件路径
    epochs=128,               # 训练轮数
    imgsz=640,                # 输入图片尺寸
    batch=24,                 # 批量大小，根据GPU内存调整
    lr0=0.01,                 # 初始学习率
    name='yolov11_WormCNN-HTT',  # 训练名称，用于保存模型
    project='runs/segment',   # 保存路径（用于分割任务）
    task='segment'            # 设置任务为分割
)


Ultralytics 8.3.36 🚀 Python-3.11.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: task=segment, mode=train, model=/data/yan/track/yolomodel/yolo11x.pt, data=./data.yaml, epochs=128, time=None, patience=100, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/segment, name=yolov11_WormCNN-HTT12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

train: Scanning /data/yan/track/datasets/train/labels... 18 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18/18 [00:00<00:00, 1454.84it/s]

train: New cache created: /data/yan/track/datasets/train/labels.cache



val: Scanning /data/yan/track/datasets/val/labels... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<00:00, 1112.72it/s]

val: New cache created: /data/yan/track/datasets/val/labels.cache


...

                   all          5         23     0.0363      0.565     0.0234     0.0154
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 2, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



102 epochs completed in 0.040 hours.
Optimizer stripped from runs/segment/yolov11_WormCNN-HTT12/weights/last.pt, 114.4MB
Optimizer stripped from runs/segment/yolov11_WormCNN-HTT12/weights/best.pt, 114.4MB

Validating runs/segment/yolov11_WormCNN-HTT12/weights/best.pt...
Ultralytics 8.3.36 🚀 Python-3.11.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 24.38it/s]


                   all          5         23      0.954      0.908       0.94       0.79
Speed: 0.1ms preprocess, 2.8ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/segment/yolov11_WormCNN-HTT12


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd9d8dc6610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
import cv2
from ultralytics import YOLO
import os

def main():
    # 加载训练好的模型
    model_path = '/data/yan/track/runs/segment/yolov11_WormCNN-HTT12/weights/best.pt'  # 替换为您的最佳模型路径
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"模型文件不存在: {model_path}")
    model = YOLO(model_path)

    # 输入视频路径
    video_path = './vedio/TS-20241123220321351.avi'  # 替换为您的视频路径
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"视频文件不存在: {video_path}")
    
    # 打开视频
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"无法打开视频文件: {video_path}")

    # 获取视频参数
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"视频帧率: {fps}, 分辨率: {width}x{height}, 总帧数: {total_frames}")

    # 定义输出视频的保存路径和编码器
    output_dir = 'output_videos'
    os.makedirs(output_dir, exist_ok=True)
    # output_path = os.path.join(output_dir, 'annotated_TS-20241123220321351.avi')
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 使用 XVID 编码器
    # out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 更换为 MP4 编码器
    output_path = os.path.join(output_dir, 'annotated_video.mp4')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        print(f"处理帧 {frame_count}/{total_frames}", end='\r')

        # 输入数据格式检查和处理
        if frame is not None:
            # 转换为 RGB 格式
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # 确保输入尺寸与模型要求一致
            frame_resized = cv2.resize(frame, (640, 640))

            # 数据归一化到 [0, 1] 范围，并确保类型为 float32
            frame_normalized = (frame_resized / 255.0).astype('float32')
        else:
            print(f"跳过空帧 {frame_count}")
            continue

        # 进行检测
        try:
            results = model.predict(source=frame_normalized, imgsz=640, stream=False)
        except Exception as e:
            print(f"处理帧 {frame_count} 时出错: {e}")
            continue

        # 绘制检测结果
        for result in results:
            annotated_frame = result.plot()

        # 转换为 BGR 格式以保存到视频
        annotated_frame_bgr = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)

        # 确保图像数据类型为 uint8
        annotated_frame_bgr_uint8 = (annotated_frame_bgr * 255).astype('uint8')

        # 写入输出视频
        out.write(annotated_frame_bgr_uint8)

    # 释放资源
    cap.release()
    out.release()
    print(f"\n处理完成，输出视频保存在: {output_path}")

if __name__ == "__main__":
    main()


视频帧率: 29.3, 分辨率: 1600x1200, 总帧数: 293
处理帧 1/293
0: 640x640 (no detections), 11.2ms
Speed: 3.2ms preprocess, 11.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
处理帧 2/293
0: 640x640 (no detections), 11.4ms
Speed: 2.3ms preprocess, 11.4ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
处理帧 3/293
0: 640x640 (no detections), 11.9ms
Speed: 2.1ms preprocess, 11.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
处理帧 4/293
0: 640x640 (no detections), 11.2ms
Speed: 2.5ms preprocess, 11.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
处理帧 5/293
0: 640x640 (no detections), 11.2ms
Speed: 2.2ms preprocess, 11.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
处理帧 6/293
0: 640x640 (no detections), 11.2ms
Speed: 2.0ms preprocess, 11.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
处理帧 7/293
0: 640x640 (no detections), 11.4ms
Speed: 2.0ms preprocess, 11.4ms inference, 0.3ms postprocess per i